In [2]:
!pip install catboost
!pip install category_encoders
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.metrics import accuracy_score, f1_score,jaccard_score,multilabel_confusion_matrix,log_loss
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

     |████████████████████████████████| 64.8MB 81kB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.read_json('https://data.cms.gov/resource/psut-35i4.json')

In [4]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,nppes_provider_zip4,nppes_provider_state,nppes_provider_country,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65,brand_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_claim_count,pdp_drug_cost,lis_claim_count,lis_drug_cost,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,...,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_bene_ge65_suppress_flg,average_age_of_beneficiaries,beneficiary_age_less_65_count,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_female_count,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,nppes_provider_street2,brand_claim_count,brand_drug_cost,other_claim_count,other_drug_cost,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,anti_psych_bene_count_ge65,nppes_provider_mi,ge65_suppress_flag,bene_count_ge65_suppress_flag,generic_suppress_flag,lis_suppress_flag,nonlis_suppress_flag,mapd_suppress_flag,pdp_suppress_flag
0,1003000126,ENKESHAFI,ARDALAN,M.D.,M,I,900 SETON DR,CUMBERLAND,21502,1854.0,MD,US,Internal Medicine,S,E,677,695.5,32639.57,14788,234.0,516.0,532.533333,23174.78,11428.0,182.0,#,552.0,5941.53,*,150.0,3032.66,527.0,29606.91,305.0,17267.72,372.0,15371.85,25.0,197.47,134.0,...,104.0,*,*,72.0,52.0,82.0,67.0,33.0,142.0,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003000142,KHALIL,RASHID,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,3536.0,OH,US,Anesthesiology,S,E,1946,2054.8,140189.01,58605,276.0,881.0,919.066667,50838.50,26089.0,160.0,NaN,1711.0,34140.73,NaN,930.0,67081.22,1016.0,73107.79,1184.0,94361.07,762.0,45827.94,1040.0,50895.21,29238.0,...,0.0,NaN,NaN,65.0,116.0,99.0,45.0,16.0,184.0,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029,SUITE 220,235.0,106048.28,0.0,0.0,200.0,37637.86,5853.0,49.0,19.23,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003000167,ESCOBAR,JULIO,DDS,M,I,5 PINE CONE RD,DAYTON,89403,7482.0,NV,US,Dentist,S,N,55,55.0,302.01,430,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*,16.0,58.19,39.0,243.82,NaN,NaN,NaN,NaN,14.0,61.95,35.0,...,22.0,NaN,NaN,72.0,NaN,NaN,NaN,NaN,16.0,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598,NaN,0.0,0.00,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,E,#,#,#,*,#,NaN,NaN
3,1003000175,REYES-VASQUEZ,BELINDA,D.D.S.,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744,4648.0,CA,US,Dentist,S,N,18,18.0,113.81,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,113.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,NaN,0.0,0.0,0.0,NaN,#,#,NaN,#,*,*,*
4,1003000282,BLAKEMORE,ROSIE,FNP,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243,1.0,TN,US,Nurse Practitioner,S,N,90,110.0,7561.21,2681,11.0,65.0,85.000000,3894.08,1931.0,NaN,NaN,62.0,984.93,NaN,22.0,485.04,68.0,7076.17,73.0,3947.56,17.0,3613.65,0.0,0.00,0.0,...,0.0,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,4.5148,3881 STEWARTS LANE,28.0,6576.28,0.0,0.0,0.0,0.00,0.0,0.0,NaN,0.0,0.0,0.0,K,NaN,*,NaN,NaN,NaN,NaN,NaN


In [5]:
df.specialty_description.unique()

array(['Internal Medicine', 'Anesthesiology', 'Dentist',
       'Nurse Practitioner', 'Family Practice', 'Obstetrics & Gynecology',
       'General Surgery', 'Urology', 'Cardiac Surgery', 'Pharmacist',
       'Physician Assistant', 'Dermatology', 'Optometry',
       'Physical Medicine and Rehabilitation', 'Radiation Oncology',
       'Infectious Disease', 'Orthopedic Surgery', 'Endocrinology',
       'Psychiatry', 'Rheumatology', 'General Practice',
       'Emergency Medicine', 'Neurology', 'Nephrology',
       'Preventive Medicine', 'Licensed Clinical Social Worker',
       'Orthopaedic Surgery', 'Ophthalmology', 'Pulmonary Disease',
       'Otolaryngology', 'Plastic and Reconstructive Surgery',
       'Cardiology', 'Psychiatry & Neurology',
       'Interventional Cardiology', 'Gastroenterology',
       'Diagnostic Radiology', 'Geriatric Medicine', 'Neurosurgery',
       'Interventional Radiology', 'Neuropsychiatry', 'Pain Management',
       'Oral Surgery (Dentist only)', 'Podiatry',

In [6]:
df['nppes_credentials']=df.nppes_credentials.str.replace(".","")
df['nppes_credentials']=df.nppes_credentials.str.replace(",","")
df['nppes_credentials']=df.nppes_credentials.str.replace("-","")
df['nppes_credentials']=df.nppes_credentials.str.replace("/","")
df['nppes_credentials']=df.nppes_credentials.str.replace(" ","")
df.nppes_credentials.nunique()

69

In [7]:

df=df[[  'nppes_credentials', 'nppes_provider_gender',
       'nppes_provider_city', 'nppes_provider_zip5',
       'nppes_provider_state', 'nppes_provider_country',
       'specialty_description', 'medicare_prvdr_enroll_status','bene_count']]

df['nppes_provider_zip5']=df['nppes_provider_zip5'].astype(str)
df.dropna(inplace=True)
df['n_patients']=pd.qcut(df.bene_count,4)

X=df.iloc[:,:-2]
y=df.iloc[:,-1]
y=pd.Series(LabelEncoder().fit_transform(y))

In [8]:
pd.Series(y).value_counts()/pd.Series(y).value_counts().sum()

0    0.255196
3    0.250577
2    0.248268
1    0.245958
dtype: float64

In [9]:
#label encoder to encode unseen values too
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)


In [10]:
#encodings and split

def onehot_all(X,y,ratio):
    
    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)
    
    obj_cols=X.select_dtypes('object').columns
    enc=ce.OneHotEncoder(cols=obj_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    return X_train, X_test,y_train, y_test

def target_all(X,y,ratio):
    
    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)
    
#     obj_cols=X.select_dtypes('object').columns
    enc=ce.TargetEncoder(handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    return X_train,X_test,y_train, y_test
    
def onehot_target(X,y,ratio,thresh):
    
    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)
    
    low_card_cols,high_card_cols=[],[]
    obj_cols=X.select_dtypes('object').columns
    for col in obj_cols:
        if X_train[col].nunique()<=thresh:
            low_card_cols.append(col)
        else:
            high_card_cols.append(col)
    
    print(low_card_cols,high_card_cols)
    
    enc=ce.OneHotEncoder(cols=low_card_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    enc=ce.TargetEncoder(cols=high_card_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    return X_train,X_test,y_train, y_test

def target_encode_multiclass(X,y,ratio):
    class_names=y.unique()
    y_classes=pd.DataFrame(columns=class_names)

    for class_ in class_names:
      y_class_=y.map(lambda x: 1 if x==class_ else 0)
      y_classes[class_]=y_class_

    X_train, X_test, y_train, y_test = train_test_split(X, y_classes, test_size=ratio, random_state=42)

    X_train_obj=X_train.select_dtypes('object')
    X_test_obj=X_test.select_dtypes('object')
    X_train=X_train.select_dtypes(exclude='object')
    X_test=X_test.select_dtypes(exclude='object')

    for class_ in class_names:
      
      enc=ce.TargetEncoder(handle_missing='return_nan').fit(X_train_obj,y_train[class_])
      temp=enc.transform(X_train_obj)
      temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
      X_train=pd.concat([X_train,temp],axis=1)
      temp=enc.transform(X_test_obj)
      temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
      X_test=pd.concat([X_test,temp],axis=1)
      
    y_train=y_train.apply(lambda row: 1 if row[1]==1 else 2 if row[2]==1 else 3 if row[3]==1 else 0, axis=1)
    y_test=y_test.apply(lambda row: 1 if row[1]==1 else 2 if row[2]==1 else 3 if row[3]==1 else 0, axis=1)

    return X_train, X_test, y_train, y_test
            
def onehot_target_encode_multiclass(X,y,ratio,thresh):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)

    low_card_cols,high_card_cols=[],[]
    obj_cols=X.select_dtypes('object').columns
    for col in obj_cols:
        if X_train[col].nunique()<=thresh:
            low_card_cols.append(col)
        else:
            high_card_cols.append(col)
    
    enc=ce.OneHotEncoder(cols=low_card_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)


    enc=ce.OneHotEncoder(handle_missing='return_nan').fit(y_train)
    y_train_wide=enc.transform(y_train)
    y_test_wide=enc.transform(y_test)


    class_names=y_train_wide.columns
    # y_classes=pd.DataFrame(columns=class_names)

    # for class_ in class_names:
    #   y_class_=y.map(lambda x: 1 if x==class_ else 0)
    #   y_classes[class_]=y_class_

    
    X_train_obj=X_train.select_dtypes('object')
    X_test_obj=X_test.select_dtypes('object')
    X_train=X_train.select_dtypes(exclude='object')
    X_test=X_test.select_dtypes(exclude='object')

    for class_ in class_names:
      
      enc=ce.TargetEncoder(handle_missing='return_nan').fit(X_train_obj,y_train_wide[class_])
      temp=enc.transform(X_train_obj)
      temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
      X_train=pd.concat([X_train,temp],axis=1)
      temp=enc.transform(X_test_obj)
      temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
      X_test=pd.concat([X_test,temp],axis=1)
      
    # y_train=y_train.apply(lambda row: 1 if row[1]==1 else 2 if row[2]==1 else 3 if row[3]==1 else 0, axis=1)
    # y_test=y_test.apply(lambda row: 1 if row[1]==1 else 2 if row[2]==1 else 3 if row[3]==1 else 0, axis=1)

    return X_train, X_test, y_train, y_test

In [11]:

def train_evaluate( X_train,X_test,y_train, y_test,cat_features=None):

  # try:
  # print("lightgbm training with gridsearch")

  # model = lgbm.LGBMClassifier(boosting_type='goss')
  # grid = GridSearchCV(estimator=model, param_grid = parameters)
  # grid.fit(X_train, y_train,verbose=0)    

  # model=grid.best_estimator_
  # evaluate(model,X_test,y_test)
  # # except: pass

  # try:
  #   print("catboost training with gridsearch")

  #   model = CatBoostClassifier(
  #       task_type="GPU", devices='0:1'
  #   )
  #   grid = GridSearchCV(estimator=model, param_grid = parameters)
  #   grid.fit(X_train, y_train,verbose=0)    

  #   model=grid.best_estimator_
  #   evaluate(model,X_test,y_test)
  # except:pass

  try:
    print("xgboost training with gridsearch")

    model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
    grid = GridSearchCV(estimator=model, param_grid = parameters_xgb)
    grid.fit(X_train, y_train,verbose=0)    

    model=grid.best_estimator_
    evaluate(model,X_test,y_test)
  except:pass

  try:  
    print("RF training with gridsearch")
    
    model=RandomForestClassifier()
    grid = GridSearchCV(estimator=model, param_grid = parameters_RF)
    grid.fit(X_train.fillna(value=0), y_train)    

    model=grid.best_estimator_
    evaluate(model,X_test.fillna(value=0),y_test)
  except:pass
    
def train_evaluate_with_cat_feat( X_train,X_test,y_train, y_test,cat_features=None):
  try:
    print("lightgbm training with gridsearch")
    for col in cat_features:
        le = LabelEncoderExt()
        le.fit(X_train[col])
        X_train[col]=le.transform(X_train[col])
        X_test[col]=le.transform(X_test[col])

    fit_params={'categorical_feature':cat_features}
    model = lgbm.LGBMClassifier(boosting_type='goss')
    grid = GridSearchCV(estimator=model, param_grid = parameters)
    grid.fit(X_train, y_train,verbose=0,**fit_params)    

    model=grid.best_estimator_
    evaluate(model,X_test,y_test)
  except:pass

  try:
    print("catboost training with gridsearch")
    fit_params={'cat_features':cat_features}
    model = CatBoostClassifier(
        task_type="GPU", devices='0:1'
    )
    grid = GridSearchCV(estimator=model, param_grid = parameters)
    grid.fit(X_train, y_train,verbose=0,**fit_params)    

    model=grid.best_estimator_
    evaluate(model,X_test,y_test)
  except:pass

In [12]:
#evaluation function

def evaluate(model,X_test,y_test):
    
    pred=model.predict(X_test)
    pred_proba=model.predict_proba(X_test)

    print('accuracy:',accuracy_score(y_test,pred))
    print('f1 macro:',f1_score(y_test,pred, average='macro'))
    # print('f1_micro:',f1_score(y_test,pred, average='micro'))
#     print(pd.Series(pred).unique())
    print('log_loss:',log_loss(y_test,pred_proba,labels=[1,2,3,0]))


In [ ]:
# X=df.iloc[:,:-1]
# y=df.iloc[:,-1:]
# X_train,X_test,y_train, y_test=onehot_all(df.iloc[:,:-1],y,.2)
# X_train,X_test,y_train, y_test=target_all(df.iloc[:,:-1],y,ratio=.2)
# X_train,X_test,y_train, y_test=onehot_target(df.iloc[:,:-1],y,ratio=.2,thresh=10)


parameters = {'depth'         : [6,8,10],
              'learning_rate' : [.01,.05,.1,.2],
              'n_estimators': [500,1000],
              # 'reg_lambda': [0,0.5,1.0]
              # 'iterations'    : [500,1000,2000]
              }
# parameters = {'depth'         : [4],
#               'learning_rate' : [.2],
#               # 'iterations'    : [50]
#               }
parameters_xgb = {
'n_estimators': [500,1000],
    # 'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [6,8,10],
    'criterion' :['gini', 'entropy'],
    # 'reg_lambda': [0,0.5,1.0]
        }

# parameters_xgb = {
# 'n_estimators': [50],
# #     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4],
# #     'criterion' :['gini', 'entropy']
#         }
parameters_RF={'max_depth':[6,8,10], 'n_estimators':[500,1000]}
# parameters_RF={'max_depth':[2], 'n_estimators':[10]}


print("##############################")
print("now with target for all")
X_train,X_test,y_train, y_test=target_all(X,y,.2)
train_evaluate( X_train,X_test,y_train, y_test)
print("##############################")
print("now with mixed for all")
X_train,X_test,y_train, y_test=onehot_target(X,y,.2,8)
train_evaluate( X_train,X_test,y_train, y_test)
print("##############################")
print("now with one hot for all")
X_train,X_test,y_train, y_test=onehot_all(X,y,.2)
train_evaluate( X_train,X_test,y_train, y_test)
print("##############################")
print("now with native cat features support")
X_train,X_test,y_train, y_test=train_test_split(X, y, test_size=.2, random_state=42)
train_evaluate_with_cat_feat(X_train,X_test,y_train, y_test,cat_features=list(X_train.select_dtypes('object').columns))


##############################
now with target for all
xgboost training with gridsearch


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: [16:36:45] /workspace/src/tree/updater_gpu_hist.cu:1407: Exception in gpu_hist: NCCL failure :unhandled cuda error /workspace/src/tree/../common/device_helpers.cuh(896)

Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x24) [0x7ff398e06cb4]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::tree::GPUHistMakerSpecialised<xgboost::detail::GradientPairInternal<double> >::Update(xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*, xgboost::DMatrix*, std::vector<xgboost::RegTree*, std::allocator<xgboost::RegTree*> > const&)+0x1270) [0x7ff3990427f0]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/lib

RF training with gridsearch
##############################
now with mixed for all
['nppes_provider_gender', 'nppes_provider_country', 'medicare_prvdr_enroll_status'] ['nppes_credentials', 'nppes_provider_city', 'nppes_provider_zip5', 'nppes_provider_state', 'specialty_description']
xgboost training with gridsearch


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: [16:37:19] /workspace/src/tree/updater_gpu_hist.cu:1407: Exception in gpu_hist: NCCL failure :unhandled cuda error /workspace/src/tree/../common/device_helpers.cuh(896)

Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x24) [0x7ff398e06cb4]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::tree::GPUHistMakerSpecialised<xgboost::detail::GradientPairInternal<double> >::Update(xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*, xgboost::DMatrix*, std::vector<xgboost::RegTree*, std::allocator<xgboost::RegTree*> > const&)+0x1270) [0x7ff3990427f0]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/lib

RF training with gridsearch


In [ ]:
proba